# Ray Serve - Model Serving Challenges

© 2019-2021, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)

Now we'll explore a nontrivial example for Ray Serve.

We'll work through an example that also covers training a model, deploying it, then updating later, based on this [documentation example](https://docs.ray.io/en/latest/serve/deployment.html). This page also has a section on [deployment to Kubernetes](https://docs.ray.io/en/latest/serve/deployment.html#deploying-as-a-kubernetes-service).

This example is from the Ray Serve [scikit-learn example.](https://docs.ray.io/en/latest/serve/tutorials/sklearn.html)
See also the Serve documentation's [mini-tutorials](https://docs.ray.io/en/latest/serve/tutorials/index.html) for using Serve with various frameworks.

In [1]:
import ray
from ray import serve
import os
import requests  # for making web requests
import tempfile

In [2]:
serve.start()

2021-10-31 17:37:21,429	INFO services.py:1252 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=94789) 2021-10-31 17:37:23,230	INFO checkpoint_path.py:15 -- Using RayInternalKVStore for controller checkpoint and recovery.
(pid=94789) 2021-10-31 17:37:23,233	INFO http_state.py:78 -- Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:yGUbcl:SERVE_PROXY_ACTOR-node:192.168.86.49-0' on node 'node:192.168.86.49-0' listening on '127.0.0.1:8000'
2021-10-31 17:37:23,480	INFO api.py:455 -- Started Serve instance in namespace 'serve'.


## Get a Model to Serve ;)

We'll begin by training a classifier with the Iris data we used before, this time using [scikit-learn](https://scikit-learn.org/stable/). The details aren't too important for our purposes, except for the fact we'll save the trained model to disk for subsequent serving.

In [3]:
import pickle
import json
import numpy as np

In [4]:
import sklearn
from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error

(pid=94791) INFO:     Started server process [94791]


In [5]:
# Load data
iris_dataset = load_iris()
data, target, target_names = iris_dataset["data"], iris_dataset[
    "target"], iris_dataset["target_names"]

In [6]:
# Instantiate model
model = GradientBoostingClassifier()

In [7]:
# Training and validation split
data, target = sklearn.utils.shuffle(data, target)
train_x, train_y = data[:100], target[:100]
val_x, val_y = data[100:], target[100:]

In [8]:
# Train and evaluate models
model.fit(train_x, train_y)
print("MSE:", mean_squared_error(model.predict(val_x), val_y))

MSE: 0.06


In [9]:
# Save the model and label to file
MODEL_PATH = os.path.join(tempfile.gettempdir(),
                          "iris_model_logistic_regression.pkl")
LABEL_PATH = os.path.join(tempfile.gettempdir(), "iris_labels.json")

In [10]:
# Save the model and label to file. (This could also be S3 or other "global" place)

with open(MODEL_PATH, "wb") as f:
    pickle.dump(model, f)
with open(LABEL_PATH, "w") as f:
    json.dump(target_names.tolist(), f)

## Create a Model and Serve It

Next, we define a servable model by instantiating a class and defining the `__call__` method that Ray Serve will use. 

In [18]:
@serve.deployment(route_prefix="/regressor")
class BoostingModel:
    def __init__(self):
        with open(MODEL_PATH, "rb") as f:
            self.model = pickle.load(f)
        with open(LABEL_PATH) as f:
            self.label_list = json.load(f)

    # async allows us to have this call concurrently            
    async def __call__(self, starlette_request):
        payload = await starlette_request.json()
        print("Worker: received starlette request with data", payload)

        input_vector = [
            payload["sepal length"],
            payload["sepal width"],
            payload["petal length"],
            payload["petal width"],
        ]
        prediction = self.model.predict([input_vector])[0]
        human_name = self.label_list[prediction]
        return {"result": human_name}

## Deploy the model

In [12]:
BoostingModel.deploy()

2021-10-31 17:37:29,350	INFO api.py:243 -- Updating deployment 'BoostingModel'. component=serve deployment=BoostingModel
(pid=94789) 2021-10-31 17:37:29,369	INFO backend_state.py:896 -- Adding 1 replicas to deployment 'BoostingModel'. component=serve deployment=BoostingModel
2021-10-31 17:37:30,097	INFO api.py:251 -- Deployment 'BoostingModel' is ready at `http://127.0.0.1:8000/regressor`. component=serve deployment=BoostingModel


## Score the model
Internally, Serve stores the model as a Ray actor and routes traffic to it as the endpoint is queried, in this case over HTTP. 

Now let’s query the endpoint to see results.

In [13]:
sample_request_input = {
    "sepal length": 1.2,
    "sepal width": 1.0,
    "petal length": 1.1,
    "petal width": 0.9,
}

We can now send HTTP requests to our route `route_prefix=/regressor` at the default port 8000

In [14]:
response = requests.get(
    "http://localhost:8000/regressor", json=sample_request_input)
print(response.text)

{
  "result": "setosa"
}
(BoostingModel pid=94788) Worker: received starlette request with data {'sepal length': 1.2, 'sepal width': 1.0, 'petal length': 1.1, 'petal width': 0.9}


In [15]:
for i in range(10):
    response = requests.get("http://localhost:8000/regressor", json=sample_request_input).json()
    print(response)

{'result': 'setosa'}
{'result': 'setosa'}
(BoostingModel pid=94788) Worker: received starlette request with data {'sepal length': 1.2, 'sepal width': 1.0, 'petal length': 1.1, 'petal width': 0.9}
{'result': 'setosa'}
{'result': 'setosa'}
{'result': 'setosa'}
{'result': 'setosa'}
{'result': 'setosa'}
{'result': 'setosa'}
{'result': 'setosa'}
{'result': 'setosa'}
(BoostingModel pid=94788) Worker: received starlette request with data {'sepal length': 1.2, 'sepal width': 1.0, 'petal length': 1.1, 'petal width': 0.9}
(BoostingModel pid=94788) Worker: received starlette request with data {'sepal length': 1.2, 'sepal width': 1.0, 'petal length': 1.1, 'petal width': 0.9}
(BoostingModel pid=94788) Worker: received starlette request with data {'sepal length': 1.2, 'sepal width': 1.0, 'petal length': 1.1, 'petal width': 0.9}
(BoostingModel pid=94788) Worker: received starlette request with data {'sepal length': 1.2, 'sepal width': 1.0, 'petal length': 1.1, 'petal width': 0.9}
(BoostingModel pid=9

## Cleanup

In [16]:
deployments = serve.list_deployments()
print(f'deployments: {deployments}')

deployments: {'BoostingModel': Deployment(name=BoostingModel,version=None,route_prefix=/regressor)}


In [17]:
serve.shutdown() 

(pid=94789) 2021-10-31 17:37:41,667	INFO backend_state.py:914 -- Removing 1 replicas from deployment 'BoostingModel'. component=serve deployment=BoostingModel


## Exercise - Try Adding more examples

Here are some things you can try:

1. Add one or more new models.